In [1]:
import os
import cv2
from keras import layers
from keras import models
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

In [2]:
path = "/kaggle/input/covid19-radiography-dataset/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset"
images = []
labels = []
i = 0
for files in os.listdir(path):
    counter = 0
    for imagesPath in os.listdir(path+"/"+files):
        img = cv2.imread(path+"/"+files+"/"+imagesPath)
        img = cv2.resize(img,(200,200))
        img = img/255
        images.append(img)
        labels.append(i)
        if counter == 1330:
            break
        counter += 1
    i += 1

In [3]:
# labels = to_categorical(labels)

In [4]:
x_train,x_test,y_train,y_test = train_test_split(images,labels,test_size=0.2,random_state=42,shuffle=True)

In [5]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [6]:
# x_train = x_train[:14812]
# x_test = x_test[:6348]
# y_train = y_train[:14812]
# y_test = y_test[:6348]

In [7]:
import numpy as np
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [8]:
from tensorflow.keras.applications import Xception
from keras_tuner import Hyperband
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt
from tensorflow.keras.applications import ResNet50

In [9]:
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(200, 200, 3))
base_model.trainable = False  # Freeze the base model layers

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [10]:
def build_model(hp):
    # Add custom classification layers
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(hp.Int('units1', min_value=256, max_value=1024, step=128), activation='relu')(x)
    x = layers.Dropout(hp.Float('dropout1', 0.2, 0.5, step=0.1))(x)
    x = layers.Dense(hp.Int('units2', min_value=128, max_value=512, step=64), activation='relu')(x)
    # x = layers.Dropout(hp.Float('dropout2', 0.2, 0.5, step=0.1))(x)
    x = layers.Dense(hp.Int('units2', min_value=64, max_value=128, step=32), activation='relu')(x)
    predictions = layers.Dense(4, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=predictions)
    
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-3, 1e-4, 1e-5])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [11]:
# Step 2: Set up the tuner
tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='hyperparam_tuning',
    project_name='xception_tuning'
)

# Step 3: Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Step 4: Perform the hyperparameter search
tuner.search(x_train, y_train, validation_data=(x_test, y_test), callbacks=[early_stopping], epochs=50)

Trial 30 Complete [00h 03m 14s]
val_accuracy: 0.4478873312473297

Best val_accuracy So Far: 0.6553990840911865
Total elapsed time: 00h 45m 29s
